### 

In [1]:
#OpenAI API Credentials einsetzen
org_key = ""
api_key = ""

# OpenAI Chat-GPT Integration

### Bibliotheken installieren und mit GIS verbinden:

In [1]:
import pandas as pd
from arcgis.features import GeoAccessor
from arcgis.gis import GIS
import os
gis = GIS("home")

/opt/conda/lib/python3.9/site-packages/arcgis/gis/__init__.py:676: UserWarning: You are logged on as t.paschke with an administrator role, proceed with caution.
  warnings.warn(


###  POI / Input Parameter definieren :

In [2]:
# Die Variable POI wurde als Eingabeparameter im Notebook definiert
POI = "Kölner Dom"

###  OpenAI Python Bibliothek installieren:

In [3]:
!pip install openai

###  Import OpenAI Bibliothek, Verbindung zur API herstellen & Unterhaltung mit ChatGPT beginnen :

In [5]:
from openai import OpenAI
client = OpenAI(organization=org_key,api_key=api_key)

In [6]:
location_raw_response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content":"Bitte ermittle mir die Adresse von "+ POI}])
location_response = location_raw_response.model_dump()
location = location_response["choices"][0]["message"]["content"].replace("\n", "")

In [7]:
description_raw_response = client.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content":"Bitte liefere eine genauere Beschreibung von " +POI}])
description_response = description_raw_response.model_dump()
description = description_response["choices"][0]["message"]["content"].replace("\n", "")

In [8]:
description

'Der Kölner Dom ist eine gotische Kathedrale und ein Wahrzeichen der Stadt Köln in Deutschland. Er befindet sich im Zentrum der Altstadt, nahe dem Rheinufer.Der Dom ist mit einer Höhe von 157 Metern eine der größten Kirchen weltweit. Seine beeindruckende Architektur ist geprägt von typischen gotischen Bauelementen wie Spitzbögen, Fialen und Maßwerkfenstern. Die Fassade besteht aus dunklem Basalt und Sandstein, was dem Gebäude ein imposantes und einzigartiges Aussehen verleiht.Im Inneren des Doms befinden sich zahlreiche kunstvolle Werke, darunter verschiedene Skulpturen, Altäre und Glasfenster. Besonders beeindruckend ist der Hochaltar, der reich mit vergoldeten Reliefs geschmückt ist. Auch die Annakapelle, der Kreuzgang und die Schatzkammer sind einen Besuch wert.Der Kölner Dom lockt jährlich Millionen von Besuchern aus aller Welt an. Es besteht die Möglichkeit, den Dom zu besichtigen und informative Führungen zu buchen. Für die Mutigen gibt es außerdem die Möglichkeit, den Domturm zu

###  DataFrame erstellen:

In [9]:
poi_df = pd.DataFrame({"POI":[POI],"Adresse":[location], "Description":[description]})
poi_df

,POI,Adresse,Description
0,Kölner Dom,Die Adresse des Kölner Doms lautet:Domkloster ...,Der Kölner Dom ist eine gotische Kathedrale un...


### Geokodieren der Adresse in einen Spatially Enabled Data Frame:

In [10]:
try:
    poi_sdf = pd.DataFrame.spatial.from_df(poi_df, address_column='Adresse')
except:
    poi_sdf = pd.DataFrame.spatial.from_df(poi_df, address_column='POI')

In [11]:
poi_sdf = poi_sdf[["POI","Adresse","Description","LongLabel","City","DisplayX","DisplayY","SHAPE"]]
poi_sdf.columns = map(str.lower, poi_sdf.columns)
poi_sdf

,poi,adresse,description,longlabel,city,displayx,displayy,shape
0,Kölner Dom,Die Adresse des Kölner Doms lautet:Domkloster ...,Der Kölner Dom ist eine gotische Kathedrale un...,"Cologne Cathedral, Domkloster 4, 50667, Cologn...",Cologne,6.95814,50.94134,"{""x"": 6.956810043424, ""y"": 50.941250062179, ""s..."


### Output Parameter definieren und Ausgabedatei für WebTool schreiben

In [15]:
# DataFrame für Ausgabe in FeatureSet umwandeln
poi_feature = poi_sdf.spatial.to_featureset()

In [16]:
# Der folgende Codeblock schreibt die Ausgabeparameterdatei für die Ausführung des Webtools.
# Achtung: Darf nur während der Ausführung des Webtools ausgeführt werden, wenn die Umgebungsvariable "ENB_JOBID" vorhanden ist!
if "ENB_JOBID" in os.environ:
    # poi_feature gibt das FeatureSet zurück und wurde als Output Parameter im Notebook definiert
    out_param_name = "poi_feature"
    out_param_file= os.path.join(
        os.environ["ENB_JOBID"], "value_" + out_param_name + ".dat")
    with open(out_param_file, "w") as writer:
        writer.write(str(poi_feature))

In [17]:
if "ENB_JOBID" in os.environ:
    # description gibt die Beschreibung als Outputparameter als Text zurück (kann z.B. im Widget angezeigt werden)
    out_param_name = "description"
    out_param_file= os.path.join(
        os.environ["ENB_JOBID"], "value_" + out_param_name + ".dat")
    with open(out_param_file, "w") as writer:
        writer.write(str(description))